## Data Preprocessing
Reference: [A data analysis and machine learning project based on data for the game Teamfight Tactics](https://github.com/rndmagtanong/ph_tft/tree/master)

Reference: [Riot Games API](https://developer.riotgames.com/apis)

In [2]:
API_KEY = "RGAPI-9bbe8343-e93f-4f90-bcb8-13a2d15586ac"
import requests

# Get the challenger players
url = "https://oc1.api.riotgames.com/tft/league/v1/challenger?queue=RANKED_TFT"
url = url + "&api_key=" + API_KEY
try:
    challengers = requests.get(url).json()
except:
    print("Error getting data from the API")

# Get names of the players
summoner_ids = [player.get("summonerId") for player in challengers.get("entries")]


In [3]:
# get puuid for each player
puuids = []
for id in summoner_ids:
    puuid_url = "https://oc1.api.riotgames.com/tft/league/v1/entries/by-summoner/" + id
    puuid_url = puuid_url + "?api_key=" + API_KEY
    try:
        puuid = requests.get(puuid_url).json()
        puuid = puuid[0].get("puuid")
        puuids.append(puuid)
    except:
        print("Error getting data from the API")

In [4]:
import pandas as pd
# get match ids for each player based on puuid
match_ids = []
for puuid in puuids:
    match_url = "https://sea.api.riotgames.com/tft/match/v1/matches/by-puuid/" + puuid + "/ids"
    # last 20 matches
    match_url = match_url + "?count=20"
    match_url = match_url + "&api_key=" + API_KEY
    try:
        match_id = requests.get(match_url).json()
        match_ids += match_id
    except:
        print("Error getting data from the API")

['OC1_643785561', 'OC1_643778472', 'OC1_643770463', 'OC1_643767414', 'OC1_643763071', 'OC1_643368627', 'OC1_643365615', 'OC1_643359924', 'OC1_643358343', 'OC1_643348646', 'OC1_643323713', 'OC1_643290437', 'OC1_643287357', 'OC1_643278451', 'OC1_643273434', 'OC1_643258030', 'OC1_643251670', 'OC1_643229050', 'OC1_643190287', 'OC1_643182438']
['OC1_643843805', 'OC1_643842210', 'OC1_643837697', 'OC1_643833812', 'OC1_643663842', 'OC1_643661259', 'OC1_643312277', 'OC1_643250956', 'OC1_643248458', 'OC1_643245764', 'OC1_643243330', 'OC1_643236889', 'OC1_643239331', 'OC1_643232946', 'OC1_643230709', 'OC1_643230379', 'OC1_643231079', 'OC1_643228406', 'OC1_643224527', 'OC1_643226235']
['OC1_643626083', 'OC1_643618761', 'OC1_643594627', 'OC1_643594189', 'OC1_643590670', 'OC1_643592069', 'OC1_643586514', 'OC1_643580970', 'OC1_643577474', 'OC1_643571760', 'OC1_643565998', 'OC1_643568016', 'OC1_643560237', 'OC1_643524498', 'OC1_643521850', 'OC1_643520480', 'OC1_643516302', 'OC1_643512666', 'OC1_643495

In [12]:
from flatten_json import flatten
# get match data for each match id
match_data = pd.DataFrame()
for match in match_ids:
    match_url = "https://sea.api.riotgames.com/tft/match/v1/matches/" + match
    match_url = match_url + "?api_key=" + API_KEY
    try:
        match = requests.get(match_url).json()
        match_resp = requests.get(match_url).json()

        flat_match_resp = flatten(match_resp)
        keyList = list(flat_match_resp.keys())

        keyList1 = [key for key in keyList if key.startswith('info_participants_0')]
        keyList2 = [key for key in keyList if key.startswith('info_participants_1')]
        keyList3 = [key for key in keyList if key.startswith('info_participants_2')]
        keyList4 = [key for key in keyList if key.startswith('info_participants_3')]
        keyList5 = [key for key in keyList if key.startswith('info_participants_4')]
        keyList6 = [key for key in keyList if key.startswith('info_participants_5')]
        keyList7 = [key for key in keyList if key.startswith('info_participants_6')]
        keyList8 = [key for key in keyList if key.startswith('info_participants_7')]

        player1info = {k:v for k, v in flat_match_resp.items() if k in keyList1}
        player2info = {k:v for k, v in flat_match_resp.items() if k in keyList2}
        player3info = {k:v for k, v in flat_match_resp.items() if k in keyList3}
        player4info = {k:v for k, v in flat_match_resp.items() if k in keyList4}
        player5info = {k:v for k, v in flat_match_resp.items() if k in keyList5}
        player6info = {k:v for k, v in flat_match_resp.items() if k in keyList6}
        player7info = {k:v for k, v in flat_match_resp.items() if k in keyList7}
        player8info = {k:v for k, v in flat_match_resp.items() if k in keyList8}

        player1 = pd.DataFrame([player1info])
        player2 = pd.DataFrame([player2info])
        player3 = pd.DataFrame([player3info])
        player4 = pd.DataFrame([player4info])
        player5 = pd.DataFrame([player5info])
        player6 = pd.DataFrame([player6info])
        player7 = pd.DataFrame([player7info])
        player8 = pd.DataFrame([player8info])

        player1.columns = player1.columns.str.replace('info_participants_0_', '')
        player2.columns = player2.columns.str.replace('info_participants_1_', '')
        player3.columns = player3.columns.str.replace('info_participants_2_', '')
        player4.columns = player4.columns.str.replace('info_participants_3_', '')
        player5.columns = player5.columns.str.replace('info_participants_4_', '')
        player6.columns = player6.columns.str.replace('info_participants_5_', '')
        player7.columns = player7.columns.str.replace('info_participants_6_', '')
        player8.columns = player8.columns.str.replace('info_participants_7_', '')

        player1 = player1.convert_dtypes()
        player2 = player2.convert_dtypes()
        player3 = player3.convert_dtypes()
        player4 = player4.convert_dtypes()
        player5 = player5.convert_dtypes()
        player6 = player6.convert_dtypes()
        player7 = player7.convert_dtypes()
        player8 = player8.convert_dtypes()

        player1 = player1.select_dtypes(exclude=['object'])
        player2 = player2.select_dtypes(exclude=['object'])
        player3 = player3.select_dtypes(exclude=['object'])
        player4 = player4.select_dtypes(exclude=['object'])
        player5 = player5.select_dtypes(exclude=['object'])
        player6 = player6.select_dtypes(exclude=['object'])
        player7 = player7.select_dtypes(exclude=['object'])
        player8 = player8.select_dtypes(exclude=['object'])

        match_data = pd.concat([match_data, player1], ignore_index = True)
        match_data = pd.concat([match_data, player2], ignore_index = True)
        match_data = pd.concat([match_data, player3], ignore_index = True)
        match_data = pd.concat([match_data, player4], ignore_index = True)
        match_data = pd.concat([match_data, player5], ignore_index = True)
        match_data = pd.concat([match_data, player6], ignore_index = True)
        match_data = pd.concat([match_data, player7], ignore_index = True)
        match_data = pd.concat([match_data, player8], ignore_index = True)
    except:
        print("Error getting data from the API")


In [15]:
# convert match data to csv and save it
match_data.to_csv("challenger.csv", index = False)
print("Data saved to challenger.csv")

Data saved to match_data.csv


## Diamond 

In [1]:
API_KEY = "RGAPI-9bbe8343-e93f-4f90-bcb8-13a2d15586ac"
import requests
# get 100 diamond IV players
diamond = "https://oc1.api.riotgames.com/tft/league/v1/entries/DIAMOND/IV?queue=RANKED_TFT&page=1"
diamond = diamond + "&api_key=" + API_KEY

try:
    diamond = requests.get(diamond).json()
except:
    print("Error getting data from the API")

# Get puuids for diamond players
diamond_puuids = [player.get("puuid") for player in diamond]

# limit to 100 players
diamond_puuids = diamond_puuids[:100]

In [2]:
# get match ids for each player based on puuid
diamond_match_ids = []
for puuid in diamond_puuids:
    match_url = "https://sea.api.riotgames.com/tft/match/v1/matches/by-puuid/" + puuid + "/ids"
    # last 20 matches
    match_url = match_url + "?count=20"
    match_url = match_url + "&api_key=" + API_KEY
    try:
        match_id = requests.get(match_url).json()
        diamond_match_ids += match_id
    except:
        print("Error getting data from the API")

In [3]:
print(diamond_match_ids)

['OC1_643779130', 'OC1_643516976', 'OC1_643517555', 'OC1_643443884', 'OC1_643438512', 'OC1_643437058', 'OC1_643426950', 'OC1_643427456', 'OC1_643344126', 'OC1_642332125', 'OC1_642318652', 'OC1_642281289', 'OC1_642277890', 'OC1_642278595', 'OC1_642221327', 'OC1_642218636', 'OC1_642216042', 'OC1_642208916', 'OC1_642205987', 'OC1_642195987', 'OC1_643931443', 'OC1_643681221', 'OC1_643611942', 'OC1_643516857', 'OC1_643514610', 'OC1_643334100', 'OC1_643232977', 'OC1_643028935', 'OC1_642909810', 'OC1_642823692', 'OC1_642820639', 'OC1_642813603', 'OC1_642811350', 'OC1_642717912', 'OC1_642625503', 'OC1_642618935', 'OC1_642518618', 'OC1_642515177', 'OC1_642410346', 'OC1_642406195', 'OC1_643593363', 'OC1_643584919', 'OC1_643584282', 'OC1_643577699', 'OC1_643572662', 'OC1_643572001', 'OC1_643482805', 'OC1_643483047', 'OC1_643292598', 'OC1_643291079', 'OC1_643280936', 'OC1_643276767', 'OC1_643105968', 'OC1_643106496', 'OC1_643107059', 'OC1_643101604', 'OC1_643101014', 'OC1_643093543', 'OC1_64309208

In [6]:

from flatten_json import flatten
import pandas as pd
API_KEY = "RGAPI-9bbe8343-e93f-4f90-bcb8-13a2d15586ac"
import requests

# get match data for each match id
diamond_match_data = pd.DataFrame()
for match in diamond_match_ids:
    match_url = "https://sea.api.riotgames.com/tft/match/v1/matches/" + match
    match_url = match_url + "?api_key=" + API_KEY
    try:
        match = requests.get(match_url).json()
        match_resp = requests.get(match_url).json()

        flat_match_resp = flatten(match_resp)
        keyList = list(flat_match_resp.keys())

        keyList1 = [key for key in keyList if key.startswith('info_participants_0')]
        keyList2 = [key for key in keyList if key.startswith('info_participants_1')]
        keyList3 = [key for key in keyList if key.startswith('info_participants_2')]
        keyList4 = [key for key in keyList if key.startswith('info_participants_3')]
        keyList5 = [key for key in keyList if key.startswith('info_participants_4')]
        keyList6 = [key for key in keyList if key.startswith('info_participants_5')]
        keyList7 = [key for key in keyList if key.startswith('info_participants_6')]
        keyList8 = [key for key in keyList if key.startswith('info_participants_7')]

        player1info = {k:v for k, v in flat_match_resp.items() if k in keyList1}
        player2info = {k:v for k, v in flat_match_resp.items() if k in keyList2}
        player3info = {k:v for k, v in flat_match_resp.items() if k in keyList3}
        player4info = {k:v for k, v in flat_match_resp.items() if k in keyList4}
        player5info = {k:v for k, v in flat_match_resp.items() if k in keyList5}
        player6info = {k:v for k, v in flat_match_resp.items() if k in keyList6}
        player7info = {k:v for k, v in flat_match_resp.items() if k in keyList7}
        player8info = {k:v for k, v in flat_match_resp.items() if k in keyList8}

        player1 = pd.DataFrame([player1info])
        player2 = pd.DataFrame([player2info])
        player3 = pd.DataFrame([player3info])
        player4 = pd.DataFrame([player4info])
        player5 = pd.DataFrame([player5info])
        player6 = pd.DataFrame([player6info])
        player7 = pd.DataFrame([player7info])
        player8 = pd.DataFrame([player8info])

        player1.columns = player1.columns.str.replace('info_participants_0_', '')
        player2.columns = player2.columns.str.replace('info_participants_1_', '')
        player3.columns = player3.columns.str.replace('info_participants_2_', '')
        player4.columns = player4.columns.str.replace('info_participants_3_', '')
        player5.columns = player5.columns.str.replace('info_participants_4_', '')
        player6.columns = player6.columns.str.replace('info_participants_5_', '')
        player7.columns = player7.columns.str.replace('info_participants_6_', '')
        player8.columns = player8.columns.str.replace('info_participants_7_', '')

        player1 = player1.convert_dtypes()
        player2 = player2.convert_dtypes()
        player3 = player3.convert_dtypes()
        player4 = player4.convert_dtypes()
        player5 = player5.convert_dtypes()
        player6 = player6.convert_dtypes()
        player7 = player7.convert_dtypes()
        player8 = player8.convert_dtypes()

        player1 = player1.select_dtypes(exclude=['object'])
        player2 = player2.select_dtypes(exclude=['object'])
        player3 = player3.select_dtypes(exclude=['object'])
        player4 = player4.select_dtypes(exclude=['object'])
        player5 = player5.select_dtypes(exclude=['object'])
        player6 = player6.select_dtypes(exclude=['object'])
        player7 = player7.select_dtypes(exclude=['object'])
        player8 = player8.select_dtypes(exclude=['object'])

        diamond_match_data = pd.concat([diamond_match_data, player1], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player2], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player3], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player4], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player5], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player6], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player7], ignore_index = True)
        diamond_match_data = pd.concat([diamond_match_data, player8], ignore_index = True)
    except:
        print("Error getting data from the API")

print(diamond_match_data)

Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error getting data from the API
Error ge

In [7]:
# convert match data to csv and save it
diamond_match_data.to_csv("diamond.csv", index = False)
print("Data saved to diamond.csv")

Data saved to diamond_match_data.csv


Manually run:
docker exec {namenode container id} hdfs dfs -put /home/nbs/diamond.csv /diamond.csv
docker exec {namenode container id} hdfs dfs -put /home/nbs/challenger.csv /challenger.csv